# Future roll scenario : Maturity Approach - Python

### Overview
Future contracts are liquid and often used to gain exposure to a variety of asset classes. However they must be rolled on a periodic basis to avoid expiry or triggering delivery.

The period surrounding contract rollover can be challenging time. Future roll strategies help to better prepare moves in the market and optimize trading positions.

In order to identify future roll optimal dates, the following methods can be used:
1. Maturity date
2. Trading volume
3. Market activity

In this example, the strategy chosen to determine the future roll date  is based on the **maturity date**.

This sample demonstrates how to request and plot from a dedicated data service **on-demand** bars for a <i>`given future, all maturities`</i>.

### Inputs/outputs
Future roll sample requires future contact's identifier and date time intervals as per inputs. It returns future's continuous price over the look back period.<br>
The results is sampled using a one day delay until front's maturity to roll from front to back. 

This sample shows also how to plot the returned bars using an open source library.
### Services used
This sample uses gRPC requests in order to retrieve information from hosted services. The queried endpoint in this script are:
* StaticData: to get the intrument identifier from the input
* Bars: to get market data for each instrument

### Modules required
1. Systemathics packages:
    * *systemathics.apis*
2. Open source packages
    * *googleapis-common-protos*
    * *protobuf*
    * *grpcio*
    * *pandas*

***

# Run future roll sample - maturity approach

### Step 1: Install packages and import them

In [ ]:
pip install googleapis-common-protos protobuf grpcio pandas matplotlib plotly systemathics.apis

In [ ]:
import os
import grpc
import pandas as pd
from datetime import datetime
import google.type.date_pb2 as date
import google.type.timeofday_pb2 as timeofday
import google.type.dayofweek_pb2 as dayofweek
import google.protobuf.duration_pb2 as duration
import google.protobuf as pb

import systemathics.apis.type.v1.memo_pb2 as memo
import systemathics.apis.type.v1.constraints_pb2 as constraints
import systemathics.apis.type.v1.dateinterval_pb2 as dateinterval
import systemathics.apis.type.v1.timeinterval_pb2 as timeinterval

import systemathics.apis.services.v1.staticdata_pb2 as staticdata
import systemathics.apis.services.v1.staticdata_pb2_grpc
import systemathics.apis.services.v1.bars_pb2 as bars
import systemathics.apis.services.v1.bars_pb2_grpc

### Step 2: Prepare API requests
The following code snippets retrieve authentication token to be used in upcomming API requests:

In [ ]:
token = f"Bearer {os.environ['AUTH0_TOKEN']}"
display(token)

### Step 3: Retrieve data

#### 3.1 Define input
In a first time the user has to decide the lookup time span and the selected instrument.<br>

In [ ]:
# Defin start and end
start = datetime(year=2018,month=2,day=1)
end = datetime(year=2018,month=7,day=31)
contract = "WBS" #  WTI Crude future

#### 3.2 Future selection
Generate the future identifier by using the [research engine](https://ganymede.systemathics.com/user-redirect/lab/tree/csharp/1-Reference%20data/symbology.ipynb). The code snippets below run a static data request to retrieve all maturities for the given instrument:

In [ ]:
# generate request and add filter values
data_request = staticdata.StaticDataRequest( asset_type = staticdata.AssetType.ASSET_TYPE_FUTURE)
data_request.future_contract.value = 'WBS'
data_request.count.value = 1000

In [ ]:
# open a gRPC channel
credentials = grpc.ssl_channel_credentials()
with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
    rpc_service = systemathics.apis.services.v1.staticdata_pb2_grpc.StaticDataServiceStub(channel)
    metadata = [('authorization', token)]
    # Process the request
    response = rpc_service.StaticData(request=data_request, metadata=metadata)
    
sorted_futures = sorted(response.futures, key=lambda x: (x.maturity.year, x.maturity.month))

In [ ]:
print("Total futures retrieved: ", len(sorted_futures))

#### 3.3 Filter needed futures

In [ ]:
# Filter futures : select only futures with a maturity matching the selected period
selected_futures = [] 
last_future = True
for future in sorted_futures:
    maturity = datetime(future.maturity.year, future.maturity.month,future.maturity.day)
    if maturity > start:
        if maturity < end:
            selected_futures.append(future)
        else:
            if last_future == True:
                selected_futures.append(future) # add the last contract
                last_future = False
            else:
                break

count = len(selected_futures)
print("Total futures selected: ", count)

#### 3.4 Prepare data for upcoming bars request: select start date, end date and Memo.

In [ ]:
from datetime import timedelta 

memos, starts, ends = [],[],[]
days_delta = 1 # number of days before rolling
begin_date = start
end_date = end
for i in range(count):
    current_future = selected_futures[i]
    maturity = datetime(current_future.maturity.year, current_future.maturity.month,current_future.maturity.day)
    # 'border' (=close to start/end dates) futures have to be dealt with differently
    if i ==0:
        # handle first future
        begin_date =start
        end_date = maturity - timedelta(days=days_delta)
        memos.append(current_future.memo)
        starts.append(begin_date)
        ends.append(end_date)
    elif i == count-1:
        # handle last future
        begin_date = end_date + timedelta(days=days_delta) #get last end date as start date +delta
        end_date = end
        memos.append(current_future.memo)
        starts.append(begin_date)
        ends.append(end_date)
    else:
        begin_date = end_date + timedelta(days=days_delta) #get last end date as start date +delta
        end_date = maturity - timedelta(days=days_delta)
        memos.append(current_future.memo)
        starts.append(begin_date)
        ends.append(end_date)
        

In [ ]:
# store in a dataframe and visualize start/end dates
d = {'Memo': memos, 'Start': starts, 'End': ends}
df = pd.DataFrame(data=d)
df.head(20) #display head to visualize output

### Step 4: Get bars to get a continuous price

#### 4.1 Define methods

In [ ]:
# Define a method that creates a request to the topology endpoint for the given ticker in order to batch request later in the sample
# Note: some secondary arguments are 'hardcoded here' to keep the method signature light:
#  - Duration (1 hour bars)
#  - Excluded Days (Sat/Sun excluded)
def get_bars_request(memo, start, end):
    
    # --> Sampling
    sampling = duration.Duration(seconds = 60 * 60)
    
    # --> Constraints : specify the start/close date, a time interval, and filtered days.

    # Set the start/close date
    start_date = date.Date(year = start.year, month =start.month, day = start.day)
    end_date = date.Date(year = end.year, month =end.month, day = end.day)
    date_interval = dateinterval.DateInterval(start_date = start_date, end_date= end_date)

    # Set days to exclude
    excluded_days = [ dayofweek.SATURDAY, dayofweek.SUNDAY ]

    # Constraints: no need time interval
    constraint = constraints.Constraints(date_intervals = [date_interval],
                                         excluded_days = excluded_days,
                                         excluded_dates = [])
    
    # Set default begin time
    begin_time = timeofday.TimeOfDay(hours = 0, minutes= 0, seconds = 0)
    
    request = bars.BarsRequest(memo = memo, sampling = sampling, begin_time = begin_time, constraints = constraint )
    return request
    

#### 4.2 Process bars request for each future
We have to use the previously defined  `get_bars_request` method to generate the request and process it. Results will be stored in a dictionary before being displayed.

In [ ]:
# Create dictionary
responses_dict = {}

for i in range(count):
    credentials = grpc.ssl_channel_credentials()
    with grpc.secure_channel("apis.systemathics.cloud:443", credentials) as channel:
        bars_service = systemathics.apis.services.v1.bars_pb2_grpc.BarsServiceStub(channel)
        metadata = [('authorization', token)]
        
        # Process the request
        request = get_bars_request(memos[i], starts[i], ends[i])
        bars_reply = bars_service.Bars(request=request, metadata=metadata)
        
        # Store reply in dictionary
        key=memos[i].ticker
        responses_dict[key] = bars_reply
display(responses_dict.keys())

### Step 5: Visualize data

The following code snippets retrieve for each maturity, the *Close prices* over the look back period from the request result and plot them with the package of your choice.<br><i>`XPlot.Plotly`</i> is used as per open source display package.

In [ ]:
import matplotlib.pyplot as plt

# create figure and axis objects with subplots()
fig,ax = plt.subplots(1,1,figsize=(25,10))
for key in responses_dict.keys():
    bars = responses_dict[key].bars
    closes = [b.close for b in bars]
    dates=[datetime.fromtimestamp(b.begin_time.seconds) for b in bars]
    ax.plot(dates, closes, marker='', linewidth=2, label=key)
ax.set_xlabel("Date",fontsize=14)
ax.set_ylabel("Close",fontsize=14)
plt.title('{0} maturity roll  - 1 hour bars'.format(contract))
plt.legend()
plt.show()